In [5]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

Xavier initialization 会根据某一层网络的输入，输出节点数量自动调整最合适的分布
下面代码，我们通过tf.random_uniform创建一个\\((-\sqrt{\frac{6}{n_{in}+n_{out}}}, \sqrt{\frac{6}{n_{in}+n_{out}}})\\)范围内的均匀分布，根据它的方差公式\\(D(x)=(max-min)^2/12\\)刚好等于\\(\frac{2}{n_{in}+n_{out}}\\),因此，下面实现的就是标准的均匀分布的Xaiver初始化器。其中fan_in是输入节点数量，fan_out是输出节点数量

In [3]:
def xavier_init(fan_in, fan_out, constant = 1):
    low = - constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform(
        shape=(fan_in, fan_out), 
        minval=low, 
        maxval=high, 
        dtype=tf.float32)

定义一个去噪自编码的class

这个class实现的功能如下（**自编码器**）
 1. 输入
 2. 隐藏层（节点数 < 输入节点数），给输入添加噪声，为的是提取到输入的某些重要输入特征
 3. 输出，（节点数 = 输入节点数）, 从隐藏层的输出的特征进行数据的恢复
 
貌似我懂了自编码器的作用了，首先，我们通过隐藏层提取到输入的特征，正是由于隐藏层的节点数少于输入节点，因此，它不可能只是对原始数据的复制，最终得到的只能是数据的**某些**特征，而如果我们能够从这些由隐藏层提取出来的特征里面恢复出来原始的数据或者是几乎能恢复原来的数据，那么我们就说，这些被提取出来的特征是有意义的（**有效特征**）。顺便说一下，计算机视觉神经网络的工作方式也是从数据中提取有效特征然后对这些特征进行进一步处理的到最终的结果的。也就是说，我们可以直接把这些隐藏层拿过来给神经网络使用，尽管很有可能把这些参数添加到神经网络里面它工作并不好，但是由于这些特征提取是**有意义的**，因此，我们不是使用随机化初始参数的方式训练网络，而是由这些与训练好的参数去初始化我们的神经网络，那么，无论怎么说，这种方式去初始化网络都要比随机初始网络参数要好用得多。因为我们不是漫无目的去初始化参数的，这些隐藏层的参数已经在我们的自编码器里面工作的很好了，那么我们也有理由相信，这些参数对于我们网络的训练也将大有帮助。

同时，这里的添加噪声类似于给数据添加正则化，也就是说提高了特征提取的精准度。

****
**因此，我们可以这样理解，自编码器是作为神经网络预训练参数的一种存在，用它预训练的参数去初始化我们神经网络的参数，将大大加快我们神经网络的迭代效率。同时也可以解决深层神经网络由于深度过深而导致的梯度爆炸和梯度消失问题。运用自编码器我们就可以不用使用人工去寻找这些特征（特征工程）**
****

这里的损失函数如下：self.x（输入），self.reconstruction（输出）
$$Loss = 0.5*\sum_{j}^{examplesSize}\sum_{i}^{nInput}(x_{i} - reconstruction_{i})^2$$

**另：softplus的函数图像如下：**
![softplus](img/softplus.png)

In [4]:
class AdditiveGaussianNoiseAutoencoder(object):
    # scale 噪声系数
    def __init__(self, n_input, n_hidden, 
                 transfer_function=tf.nn.softplus, 
                 optimizer=tf.train.AdamOptimizer(),
                 scale = 0.1):
        # 这个是输入的节点数
        self.n_input = n_input
        # 这个是中间隐藏层的节点数
        self.n_hidden = n_hidden
        # 这里的transfer作为激活函数，这里的tf.nn.softplus是类似于ReLU函数的平滑实现
        self.transfer = transfer_function
        # 一个占位符，作用是噪声系数
        self.scale = tf.placeholder(dtype=tf.float32)
        # 这个是作为噪声系数的参数提供者
        self.train_scale = scale
        # 这个里面存储着神经网络的参数
        network_weights = self._initialize_weights()
        self.weights = network_weights
        # 定义网络结构，x是作为神经网络的输入
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        # hidden 是作为隐藏层网络的隐藏层，这里默认的是使用softplus作为激活函数
        # 这是一个能够提取特征的隐藏层
        self.hidden = self.transfer(
            tf.add(
                # 这里的scale * tf.random_normal((n_input,))的目的就是给输入添加噪声
                tf.matmul(self.x + scale * tf.random_normal((n_input,)),
                          self.weights['w1']),
                self.weights['b1']
            )
        )
        # 通过隐藏层的输出来重新构造输出，这也就是自编码器的核心所在
        # 即首先通过一个节点数少于输入的隐藏层编码输入后输出
        # 然后对上一层的输入重构输入，也就是数据的重建工作
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
        # 定义损失函数
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        # 定义最小化损失函数的操作
        self.optimizer = optimizer.minimize(self.cost)
        # 初始化神经网络参数
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
    
    # 初始化神经网络参数，这里的参数第一个隐藏层使用的是xavier参数初始化，第二层就直接使用了zeros来进行初始化
    # 这里使用了字典dict来进行参数的保存，优点是可以根据名称来直接查看到具体的参数
    # 因此，使用字典也是一个切实可行的方法，我之前定义神经网络是使用的layer的方式，把参数绑定在特定的layer上面
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype=tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        return all_weights
    
    # 根据提供的数据X来查看我们网络的损失函数数值，并同时运行优化算法
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict = {self.x:X, self.scale:self.train_scale})
        return cost
    
    # 这个就只是根据输入来查看损失函数数值了，并么有运行优化算法
    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict={self.x : X, self.scale:self.train_scale})
    
    # 这里是查看隐藏层的输出
    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict = {self.x : X, self.scale : self.train_scale})
    
    # 根据我们提供的编码的隐藏层数据，来生成我们的原始数据，如果我们没有提供，那么就默认生成个随机化的隐藏层数据
    def generate(self, hidden = None):
        if hidden is None:
            hidden = np.random.normal(size=self.weights['b1'])
        return self.sess.run(self.reconstruction, feed_dict={self.hidden:hidden})
    
    # 根据我们提供的数据X，计算自编码器的输出
    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x : X, self.scale : self.train_scale})
    
    # 获取隐藏层w参数
    def getWeights(self):
        return self.sess.run(self.weights['w1'])
    
    # 获取隐藏层b参数
    def getBiases(self):
        return self.sess.run(self.weight['b1'])

从上面的代码给我的感觉来看，这个代码只有某些地方和单隐层softmax函数有些许的差别。
1. 最后一层网络**没有应用激活函数**
2. 隐藏层在初始化w参数的时候运用了xavier算法，感觉这个不能算是差别，softmax深层神经网络感觉如果应用xavier算法的应该也不错
3. 损失函数，损失函数不同也正常，毕竟不同的算法应用的损失函数很大几率上不会相同
4. **最主要的不同如下：**
 ```python
    tf.matmul(self.x + scale * tf.random_normal((n_input,)),
                          self.weights['w1']),
                self.weights['b1'] 

 ```
 这里主要有一个给输入添加噪声的操作，这个在一般的神经网络里面不会出现，尽管说在某些神经网络里面会有正则化的操作，但是正则化一般都是在数据  整体做的或者是在损失函数和参数调整上面（L2）以及在节点输出上面（dropout）。

导入MNIST数据集

In [6]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True, dtype=tf.float32)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


对训练和测试的数据进行标准化处理，即让数据变成均值为0，标准差为1的分布。

这里必须保证训练测试数据都使用完全相同的Scaler，这样才能保证后面模型处理的一致性。

In [7]:
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test

定义一个获取随机block数据的函数，取一个从0到len(data)-batch_size之间的随机整数，再以这个随机数作为block的起始位置然后顺序取到一个batch-size的数据。这输入不放回抽样，可以提高数据的利用效率。

In [8]:
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]

使用standard_scale对训练集和测试集进行标准化转换

In [9]:
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)

定义几个常用的参数，总训练样本数，最大训练轮数（epoch）设为200， batch_size设为128，设置每隔五轮就显示一次损失cost

In [17]:
n_samples = int(mnist.train.num_examples)
training_epochs = 40
batch_size = 128
display_step = 5

创建一个AGN自编码器实例，参数如下：
+ 输入节点数 = 784
+ 隐藏层节点数 = 200
+ 隐藏层的激活函数为softplus
+ 优化器为Adam
+ 学习速率 = 0.001
+ 噪声系数 = 0.01

In [12]:
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input = 784,
                                              n_hidden = 200,
                                              transfer_function = tf.nn.softplus,
                                              optimizer = tf.train.AdamOptimizer(learning_rate=0.001),
                                              scale = 0.01)

In [18]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
    
    if epoch % display_step == 0:
        print("Epoch:", "%04d"%(epoch+1), "cost=", "{:.9f}".format(avg_cost))

Epoch: 0001 cost= 7528.618572159
Epoch: 0006 cost= 7628.468978977
Epoch: 0011 cost= 7924.401267045
Epoch: 0016 cost= 7511.852602273
Epoch: 0021 cost= 7428.076739773
Epoch: 0026 cost= 6953.504581250
Epoch: 0031 cost= 7548.976129545
Epoch: 0036 cost= 7067.744722727
Epoch: 0041 cost= 6818.936491477
Epoch: 0046 cost= 7441.901722727
Epoch: 0051 cost= 7594.003073295
Epoch: 0056 cost= 7837.442783523
Epoch: 0061 cost= 6869.249369318
Epoch: 0066 cost= 7082.513166477
Epoch: 0071 cost= 7504.731242045
Epoch: 0076 cost= 7796.522136932
Epoch: 0081 cost= 7229.429959091
Epoch: 0086 cost= 7350.870432386
Epoch: 0091 cost= 7133.561738636
Epoch: 0096 cost= 7335.148955114
Epoch: 0101 cost= 7253.485199432
Epoch: 0106 cost= 6996.788428409
Epoch: 0111 cost= 7010.632452273
Epoch: 0116 cost= 7012.268700568
Epoch: 0121 cost= 7115.011426136
Epoch: 0126 cost= 7112.795385795
Epoch: 0131 cost= 7288.342188636
Epoch: 0136 cost= 7126.340212500
Epoch: 0141 cost= 7175.015500000
Epoch: 0146 cost= 6918.712265909
Epoch: 015

In [21]:
print("Total cost:" + str(autoencoder.calc_total_cost(X_test)))

Total cost:606271.0


现在无监督预训练的使用场景比以前少了许多，训练全连接的MLP或者CNN，RNN时，我们都不需要先使用无监督训练提取特征。但是无监督学习乃至AutoEncoder依然是有用的，无监督学习也将是深度学习接下来的一个重要发展方向。

In [ ]:
autoencoder.sess.close()